# **Dependency**

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
!pip -q install ultralytics opencv-python-headless pandas numpy plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00


# **Upload Video**

In [ ]:
import io, os, json
from google.colab import files
from IPython.display import HTML
import cv2

# 1) Select and upload a 10–15 second video
print("Please select your 10–15 second video file (mp4/mov/mkv/avi):")
up = files.upload()  # Pop up file selection window
assert len(up) > 0, "No file selected"
video_path = list(up.keys())[0]
print(f"[OK] Video uploaded: {video_path}")

Please select your 10–15 second video file (mp4/mov/mkv/avi):


Saving play_001_fullplay_Corner__Speed_Out___Curl__Cross__Flat-R.mov to play_001_fullplay_Corner__Speed_Out___Curl__Cross__Flat-R.mov
[OK] Video uploaded: play_001_fullplay_Corner__Speed_Out___Curl__Cross__Flat-R.mov


# **Read & Merge**

In [1]:
file_tracking = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/tracking_gameId_2017090700.csv"
file_games    = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/games.csv"
file_plays    = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/plays.csv"
file_players    = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/players.csv"
tracking = pd.read_csv(file_tracking)
games    = pd.read_csv(file_games)
plays    = pd.read_csv(file_plays)
players  = pd.read_csv(file_players)


rename_map = {}
if "frame.id" in tracking.columns:
    rename_map["frame.id"] = "frameId"
if "displayName" not in tracking.columns and "player" in tracking.columns:
    rename_map["player"] = "displayName"
if rename_map:
    tracking = tracking.rename(columns=rename_map)
if "event" not in tracking.columns:
    tracking["event"] = np.nan

df = tracking.merge(games, on="gameId", how="inner").merge(plays, on=["gameId","playId"], how="inner")

NameError: name 'pd' is not defined

# **Select Play**

In [ ]:
GAME_ID = 2017090700
PLAY_ID = 1376
example = df[(df["gameId"]==GAME_ID) & (df["playId"]==PLAY_ID)].copy()
assert len(example) > 0, "No play found, please check gameId / playId."

# **Stable ID**

In [ ]:
def entity_id(row):
    t = str(row.get("team", "")).lower()
    if t == "football":
        return "football"
    nid = row.get("nflId", np.nan)
    return f"p{int(nid)}" if pd.notna(nid) else f"player:{row.get('displayName', 'NA')}"
example["entity"] = example.apply(entity_id, axis=1)

# **Field Style**

In [ ]:
def field_shapes_matplotlib_style():
    shapes = []
    # Grass
    shapes.append(dict(type="rect", x0=0, x1=120, y0=0, y1=53.3,
                       line=dict(width=0), fillcolor="darkgreen", layer="below"))
    # End zones
    shapes.append(dict(type="rect", x0=0,   x1=10,  y0=0, y1=53.3,
                       line=dict(width=0), fillcolor="rgba(220,20,60,0.85)", layer="below"))
    shapes.append(dict(type="rect", x0=110, x1=120, y0=0, y1=53.3,
                       line=dict(width=0), fillcolor="rgba(255,69,0,0.85)", layer="below"))
    # 10-yard lines
    for x in range(0, 121, 10):
        shapes.append(dict(type="line", x0=x, x1=x, y0=0, y1=53.3,
                           line=dict(color="white", width=2), layer="below"))
    # Four sets of short ticks
    short_ticks = [(0.4, 0.7), (52.5, 53.0), (22.91, 23.57), (29.73, 30.39)]
    for x in range(11, 110):
        for y0, y1 in short_ticks:
            shapes.append(dict(type="line", x0=x, x1=x, y0=y0, y1=y1,
                               line=dict(color="white", width=1), layer="below"))
    # Outer border
    shapes.append(dict(type="rect", x0=0, x1=120, y0=0, y1=53.3,
                       line=dict(color="white", width=2), fillcolor="rgba(0,0,0,0)", layer="below"))
    return shapes

def field_annotations_matplotlib_style():
    ann = []
    # Yardage numbers (two-way)
    for x in range(20, 110, 10):
        number = x if x <= 50 else 120 - x
        ann.append(dict(x=x, y=5, text=str(number - 10), showarrow=False,
                        font=dict(size=20, color="white"), xanchor="center", yanchor="middle"))
        ann.append(dict(x=x - 0.95, y=53.3 - 5, text=str(number - 10), showarrow=False,
                        font=dict(size=20, color="white"), textangle=180,
                        xanchor="center", yanchor="middle"))
    # "TD" added only once (moved out of loop to avoid duplication)
    ann.append(dict(
        x=5, y=26.65, text="TD", showarrow=False,
        font=dict(size=52, color="white", family="Arial Black"),
        textangle=90, xanchor="center", yanchor="middle", opacity=0.95
    ))
    ann.append(dict(
        x=115, y=26.65, text="TD", showarrow=False,
        font=dict(size=52, color="white", family="Arial Black"),
        textangle=-90, xanchor="center", yanchor="middle", opacity=0.95
    ))
    return ann

# **Colors**

In [ ]:
def first_available_col(df, candidates, default):
    for c in candidates:
        if c in df.columns and len(df[c]) > 0:
            return df[c].iloc[0]
    return default

home_abbr = first_available_col(example, ["homeTeamAbbr"], "HOME")
away_abbr = first_available_col(example, ["visitorTeamAbbr", "awayTeamAbbr"], "AWAY")

team_color_map = {
    "home": "#FFD700",     # Gold
    "away": "#00BFFF",     # Blue
    "football": "#8B4513"  # Brown Football
}

example["team_low"] = example["team"].astype(str).str.lower()

# More robust "football" detection (more intuitive fallback)
if "displayName" in example:
    display_lower = example["displayName"].astype(str).str.lower()
else:
    display_lower = pd.Series("", index=example.index)

if "position" in example:
    position_lower = example["position"].astype(str).str.lower()
else:
    position_lower = pd.Series("", index=example.index)

is_ball = (
    example["team_low"].eq("football")
    | display_lower.isin(["football", "ball"])
    | position_lower.eq("football")
)
example.loc[is_ball, "team_low"] = "football"

# **Plotting**

In [ ]:
fig = px.scatter(
    example, x="x", y="y",
    animation_frame="frameId",
    animation_group="entity",
    color="team_low",
    color_discrete_map=team_color_map,
    hover_name="displayName",
    hover_data={"x":":.2f","y":":.2f","nflId":True,"team_low":True},
    range_x=[0, 120], range_y=[-5, 58.3],
    height=640,
    title=f"{away_abbr} at {home_abbr} — gameId={GAME_ID}, playId={PLAY_ID}"
)

fig.update_yaxes(scaleanchor="x", scaleratio=1, showgrid=False, zeroline=False)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_layout(
    shapes=field_shapes_matplotlib_style(),
    annotations=field_annotations_matplotlib_style(),
    legend_title_text="Team",
    margin=dict(l=20, r=20, t=60, b=20),
    plot_bgcolor="white", paper_bgcolor="white"
)
fig.update_traces(marker=dict(size=12, opacity=0.95, line=dict(width=1, color="white")))

fig.show()
fig.write_html("nfl_play_animation_matplotlib_style_ball_fixed_no_numbers.html")



In [ ]:
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.4 MB/s eta 0:00:00


# **Visual**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from ipywidgets import interact, Button, HBox, Output, IntText, VBox
from IPython.display import display

# ============================================================
# LOAD ALL DATA
# ============================================================
file_tracking = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/tracking_gameId_2017090700.csv"
file_games    = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/games.csv"
file_plays    = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/plays.csv"
file_players  = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/players.csv"

tracking = pd.read_csv(file_tracking)
games    = pd.read_csv(file_games)
plays    = pd.read_csv(file_plays)
players  = pd.read_csv(file_players)

# Normalize tracking column names
rename_map = {}
if "frame.id" in tracking.columns:
    rename_map["frame.id"] = "frameId"
if "displayName" not in tracking.columns and "player" in tracking.columns:
    rename_map["player"] = "displayName"
if rename_map:
    tracking = tracking.rename(columns=rename_map)

if "event" not in tracking.columns:
    tracking["event"] = np.nan

# ============================================================
# FILTER TO PASS PLAYS + OFFENSE + RECEIVERS ONLY
# ============================================================
# Pass plays: PassLength or PassResult not null
pass_plays = plays[(plays["PassLength"].notna()) | (plays["PassResult"].notna())]

# Receiver positions we care about
receiver_positions = ["WR", "TE", "RB", "FB", "HB"]
receivers = players[players["PositionAbbr"].isin(receiver_positions)][["nflId", "PositionAbbr"]]

# Merge tracking + games + pass_plays
df = (
    tracking
    .merge(games, on="gameId", how="inner")
    .merge(pass_plays, on=["gameId", "playId"], how="inner")
)

# Map tracking 'team' (home/away) -> actual team abbr
def map_team(row):
    if row["team"] == "home":
        return row["homeTeamAbbr"]
    elif row["team"] == "away":
        return row["visitorTeamAbbr"]
    return None

df["teamAbbr"] = df.apply(map_team, axis=1)

# Keep only offense
df = df[df["teamAbbr"] == df["possessionTeam"]]

# Keep only receivers
df = df.merge(receivers, on="nflId", how="inner")

# ============================================================
# B) COMPUTE ROUTE FEATURES PER (gameId, playId, nflId)
#    - start at SNAP (ball_snap)
#    - end at PASS (pass_forward) or last frame
# ============================================================
def compute_route_features(df_in: pd.DataFrame) -> pd.DataFrame:
    sub = df_in[["gameId", "playId", "nflId", "frameId", "x", "y", "event"]].copy()

    # ball_snap per play
    snap_frames = (
        sub[sub["event"] == "ball_snap"]
        .groupby(["gameId", "playId"])["frameId"]
        .min()
        .rename("snapFrame")
        .reset_index()
    )

    # pass_forward per play
    pass_frames = (
        sub[sub["event"] == "pass_forward"]
        .groupby(["gameId", "playId"])["frameId"]
        .min()
        .rename("passFrame")
        .reset_index()
    )

    # max frame per play (fallback if no pass_forward)
    max_frame = (
        sub.groupby(["gameId", "playId"])["frameId"]
        .max()
        .rename("maxFrame")
        .reset_index()
    )

    sub = sub.merge(snap_frames, on=["gameId", "playId"], how="left")
    sub = sub.merge(pass_frames, on=["gameId", "playId"], how="left")
    sub = sub.merge(max_frame,   on=["gameId", "playId"], how="left")

    # drop pre-snap frames when snapFrame is known
    sub = sub[(sub["snapFrame"].isna()) | (sub["frameId"] >= sub["snapFrame"])]

    # routeEndFrame = passFrame if exists else maxFrame
    sub["routeEndFrame"] = np.where(sub["passFrame"].notna(),
                                    sub["passFrame"],
                                    sub["maxFrame"])

    # start frame per player AFTER snap (since we filtered)
    start_frame = (
        sub.groupby(["gameId", "playId", "nflId"])["frameId"]
        .min()
        .rename("startFrame")
        .reset_index()
    )
    sub = sub.merge(start_frame, on=["gameId", "playId", "nflId"], how="left")

    # start position at startFrame
    start_pos = sub[sub["frameId"] == sub["startFrame"]][
        ["gameId", "playId", "nflId", "x", "y"]
    ].rename(columns={"x": "x_start", "y": "y_start"})

    # end position at routeEndFrame
    end_key = sub[["gameId", "playId", "nflId", "routeEndFrame"]].drop_duplicates()
    end_key = end_key.rename(columns={"routeEndFrame": "frameId"})

    end_pos = end_key.merge(
        sub[["gameId", "playId", "nflId", "frameId", "x", "y"]],
        on=["gameId", "playId", "nflId", "frameId"],
        how="left"
    ).rename(columns={"x": "x_end", "y": "y_end"})

    route = start_pos.merge(end_pos, on=["gameId", "playId", "nflId"], how="inner")

    route["dx"] = route["x_end"] - route["x_start"]
    route["dy"] = route["y_end"] - route["y_start"]
    route["dist"] = np.sqrt(route["dx"]**2 + route["dy"]**2)
    route["angle_deg"] = np.degrees(np.arctan2(route["dy"], route["dx"]))

    return route[["gameId", "playId", "nflId", "dx", "dy", "dist", "angle_deg"]]

# Helper for side
def side_sign(dy):
    if dy > 0:
        return "L"
    if dy < 0:
        return "R"
    return "C"

def label_route(row):
    """
    Map (dx, dy, dist, angle) to one of:
    Block and Go, Comeback, Corner, Cross, Curl, Deep Out, Dig,
    Double Move, Drag, Flat, Flat-L, Flat-R, Go, Hitch, In, Out,
    Over, Post, Screen, Screen Block, Screen-L, Screen-R, Sit,
    Slant, Slide, Speed Out, Wheel
    """
    dx = row["dx"]
    dy = row["dy"]
    d  = row["dist"]
    a  = row["angle_deg"]     # [-180, 180]
    side = side_sign(dy)
    abs_a = abs(a)

    # --------------------------
    # Screens / very short
    # --------------------------
    if d < 2 and dx <= 1:
        if side != "C":
            return "Screen Block"
        return "Sit"

    if d < 4 and dx >= 0:
        if side == "L":
            return "Screen-L"
        if side == "R":
            return "Screen-R"
        return "Screen"

    # --------------------------
    # Vertical family
    # --------------------------
    vertical = (-20 <= a <= 20)
    comeback = (a >= 150 or a <= -150)

    if vertical and d >= 12:
        return "Go"

    if comeback and d >= 8:
        return "Comeback"

    if vertical and 4 <= d < 8:
        return "Hitch"

    if vertical and d < 4:
        return "Sit"

    # --------------------------
    # Lateral / shallow horizontal
    # --------------------------
    if d < 7 and abs_a >= 60:
        if side == "L":
            return "Flat-L"
        if side == "R":
            return "Flat-R"
        return "Flat"

    if 4 <= d < 8 and abs_a > 70 and abs(dx) < 3:
        return "Slide"

    # --------------------------
    # In/Out/Slant/Speed Out/Deep Out
    # --------------------------
    if 20 < abs_a <= 60 and d < 7:
        return "Speed Out" if abs_a >= 35 else "Out"

    if 20 < abs_a <= 60 and 7 <= d < 12:
        return "Out"

    if 60 < abs_a <= 110 and d >= 10:
        return "Deep Out"

    if 20 < abs_a <= 45 and d < 7:
        return "Slant"

    if 20 < abs_a <= 45 and 7 <= d < 12:
        return "In"

    if 20 < abs_a <= 45 and d >= 12:
        return "Dig"

    # --------------------------
    # Cross / Over / Drag
    # --------------------------
    if 10 < abs_a <= 35 and d >= 10:
        return "Over"

    if 10 < abs_a <= 35 and d < 10:
        return "Cross"

    if abs_a < 10 and d < 10 and abs(dy) > 3:
        return "Drag"

    # --------------------------
    # Corner / Post
    # --------------------------
    if 35 < abs_a <= 65 and d >= 10:
        return "Corner"

    if 45 < abs_a <= 80 and d >= 12:
        return "Post"

    # --------------------------
    # Wheel
    # --------------------------
    if d >= 10 and abs(dy) > 5 and abs_a < 45:
        return "Wheel"

    # --------------------------
    # Curl: comeback-ish but not as deep
    # --------------------------
    if comeback and d < 8:
        return "Curl"

    # Fallback: Double Move
    return "Double Move"


# Compute route features and labels
route_features = compute_route_features(df)

# Distance-based route flag
MIN_ROUTE_DIST = 3.0  # yards
route_features["isRoute"] = route_features["dist"] >= MIN_ROUTE_DIST

route_features["routeLabel"] = route_features.apply(label_route, axis=1)

# Merge labels into main df
for c in ["dx", "dy", "dist", "angle_deg", "routeLabel", "isRoute"]:
    if c in df.columns:
        df.drop(columns=c, inplace=True)

df = df.merge(
    route_features[["gameId", "playId", "nflId", "dx", "dy", "dist",
                    "angle_deg", "routeLabel", "isRoute"]],
    on=["gameId", "playId", "nflId"],
    how="left"
)

print("routeLabel in df:", "routeLabel" in df.columns)
print("Route label counts (all):")
print(df["routeLabel"].value_counts().head())

print("\nAverage #routes/play (dist >= 3):")
routes_per_play = (
    route_features[route_features["isRoute"]]
    .groupby(["gameId", "playId"])["nflId"]
    .nunique()
)
print(routes_per_play.describe())

# ============================================================
# FIELD DRAWING HELPERS — SAME STYLE AS ORIGINAL CODE
# ============================================================
def field_shapes_matplotlib_style():
    shapes = []
    # Grass
    shapes.append(dict(type="rect", x0=0, x1=120, y0=0, y1=53.3,
                       line=dict(width=0), fillcolor="darkgreen", layer="below"))
    # End zones
    shapes.append(dict(type="rect", x0=0,   x1=10,  y0=0, y1=53.3,
                       line=dict(width=0), fillcolor="rgba(220,20,60,0.85)", layer="below"))
    shapes.append(dict(type="rect", x0=110, x1=120, y0=0, y1=53.3,
                       line=dict(width=0), fillcolor="rgba(255,69,0,0.85)", layer="below"))

    # 10-yard lines
    for x in range(0, 121, 10):
        shapes.append(dict(type="line", x0=x, x1=x, y0=0, y1=53.3,
                           line=dict(color="white", width=2), layer="below"))

    # Short ticks at the hashes
    short_ticks = [(0.4, 0.7), (52.5, 53.0), (22.91, 23.57), (29.73, 30.39)]
    for x in range(11, 110):
        for y0, y1 in short_ticks:
            shapes.append(dict(type="line", x0=x, x1=x, y0=y0, y1=y1,
                               line=dict(color="white", width=1), layer="below"))

    # Outer border
    shapes.append(dict(type="rect", x0=0, x1=120, y0=0, y1=53.3,
                       line=dict(color="white", width=2),
                       fillcolor="rgba(0,0,0,0)", layer="below"))
    return shapes


def field_annotations_matplotlib_style():
    ann = []
    # Yard numbers
    for x in range(20, 110, 10):
        number = x if x <= 50 else 120 - x
        ann.append(dict(
            x=x, y=5, text=str(number - 10), showarrow=False,
            font=dict(size=20, color="white"),
            xanchor="center", yanchor="middle"
        ))
        ann.append(dict(
            x=x - 0.95, y=53.3 - 5, text=str(number - 10), showarrow=False,
            font=dict(size=20, color="white"), textangle=180,
            xanchor="center", yanchor="middle"
        ))

    # TD labels
    ann.append(dict(
        x=5, y=26.65, text="TD", showarrow=False,
        font=dict(size=52, color="white", family="Arial Black"),
        textangle=90, xanchor="center", yanchor="middle", opacity=0.95
    ))
    ann.append(dict(
        x=115, y=26.65, text="TD", showarrow=False,
        font=dict(size=52, color="white", family="Arial Black"),
        textangle=-90, xanchor="center", yanchor="middle", opacity=0.95
    ))
    return ann

# ============================================================
# BUILD A ROUTE CATALOG (ONE ROW PER ROUTE)
# ============================================================
real_routes = route_features[route_features["isRoute"]].copy()

route_catalog = (
    real_routes
    .merge(
        df[[
            "gameId", "playId", "nflId",
            "displayName", "homeTeamAbbr", "visitorTeamAbbr"
        ]].drop_duplicates(),
        on=["gameId", "playId", "nflId"],
        how="left"
    )
    .sort_values(["gameId", "playId", "nflId"])
    .reset_index(drop=True)
)

print("Total individual routes:", len(route_catalog))
print(route_catalog[["gameId", "playId", "nflId", "displayName", "routeLabel"]].head())

# ============================================================
# DRAW A SINGLE ROUTE (ONE PLAYER ONLY)
# ============================================================
def draw_route(route_idx: int):
    """
    Visualize a single route (one gameId, playId, nflId) over time.
    route_idx indexes into route_catalog.
    """
    if route_idx < 0 or route_idx >= len(route_catalog):
        # Handle out of bounds by resetting to valid range
        route_idx = max(0, min(route_idx, len(route_catalog) - 1))
        route_idx_widget.value = route_idx # Update the IntText widget if it's out of bounds

    r = route_catalog.iloc[route_idx]
    g   = int(r.gameId)
    p   = int(r.playId)
    nid = int(r.nflId)
    name  = r.displayName
    label = r.routeLabel
    home  = r.homeTeamAbbr
    away  = r.visitorTeamAbbr

    # subset: only this receiver on this play
    sub = df[(df["gameId"] == g) & (df["playId"] == p) & (df["nflId"] == nid)].copy()
    if sub.empty:
        plot_output.clear_output()
        with plot_output:
            print(f"No tracking rows for route_idx={route_idx}, gameId={g}, playId={p}, nflId={nid}")
        return

    fig = px.scatter(
        sub,
        x="x", y="y",
        animation_frame="frameId",
        animation_group="nflId",
        color="routeLabel",   # single route type but keeps consistent legend
        hover_name="displayName",
        hover_data={
            "routeLabel": True,
            "dist": ":.2f" if "dist" in sub.columns else True,
            "x":":.2f",
            "y":":.2f",
            "nflId": True
        },
        range_x=[0, 120],
        range_y=[-5, 58.3],
        height=640,
        title=f"{away} at {home} — gameId={g}, playId={p} — {name} ({nid}) — {label}"
    )

    fig.update_yaxes(scaleanchor="x", scaleratio=1, showgrid=False, zeroline=False)
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_layout(
        shapes=field_shapes_matplotlib_style(),
        annotations=field_annotations_matplotlib_style(),
        legend_title_text="Route",
        margin=dict(l=20, r=20, t=60, b=20),
        plot_bgcolor="white",
        paper_bgcolor="white"
    )
    fig.update_traces(marker=dict(size=12, opacity=0.95,
                                  line=dict(width=1, color="white")))
    plot_output.clear_output(wait=True)
    with plot_output:
        display(fig)

# ============================================================
# ROUTE TOGGLER — ONE ROUTE AT A TIME (with buttons)
# ============================================================
max_routes = len(route_catalog)
current_route_idx = 0

# Output widget for the plot
plot_output = Output()

# Text input for index
route_idx_widget = IntText(
    value=current_route_idx,
    description='Route Index:',
    min=0,
    max=max_routes - 1,
    disabled=False
)

def on_index_change(change):
    global current_route_idx
    new_idx = change.new
    if 0 <= new_idx < max_routes:
        current_route_idx = new_idx
        draw_route(current_route_idx)
    else:
        route_idx_widget.value = current_route_idx # Revert if invalid input

route_idx_widget.observe(on_index_change, names='value')

# Buttons
prev_button = Button(description="Previous")
next_button = Button(description="Next")

def on_prev_button_clicked(b):
    global current_route_idx
    if current_route_idx > 0:
        current_route_idx -= 1
        route_idx_widget.value = current_route_idx

def on_next_button_clicked(b):
    global current_route_idx
    if current_route_idx < max_routes - 1:
        current_route_idx += 1
        route_idx_widget.value = current_route_idx

prev_button.on_click(on_prev_button_clicked)
next_button.on_click(on_next_button_clicked)

print("Use the buttons or type the index to step through individual routes.")
print(f"Total routes: {max_routes}")

display(HBox([prev_button, next_button, route_idx_widget]))
display(plot_output)

# Initial plot display
draw_route(current_route_idx)


routeLabel in df: True
Route label counts (all):
routeLabel
Double Move    7432
Comeback       6954
Deep Out       4400
Dig            2811
Out            2428
Name: count, dtype: int64

Average #routes/play (dist >= 3):
count    79.000000
mean      4.911392
std       0.285992
min       4.000000
25%       5.000000
50%       5.000000
75%       5.000000
max       5.000000
Name: nflId, dtype: float64
Total individual routes: 388
       gameId  playId      nflId     displayName   routeLabel
0  2017090700      68   497240.0  Rob Gronkowski  Double Move
1  2017090700      68  2530515.0     Chris Hogan     Comeback
2  2017090700      68  2533046.0    Dwayne Allen  Double Move
3  2017090700      68  2539265.0    Rex Burkhead     Deep Out
4  2017090700      68  2543498.0   Brandin Cooks     Comeback
Use the buttons or type the index to step through individual routes.
Total routes: 388


Output()

# **Model**

In [ ]:
import pandas as pd

MANUAL_LABEL_XLSX = "/content/drive/MyDrive/GenAI_Final/ManualLabels.xlsx"
labels_df = pd.read_excel(MANUAL_LABEL_XLSX)


In [ ]:
# ============================================================
# 1) LOAD MANUAL LABELS (FROM /content/ManualLabels.xlsx, INDEX-BASED)
# ============================================================
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

_raw = pd.read_excel(MANUAL_LABEL_XLSX, header=None)

labels = _raw.iloc[4:].copy()
labels = labels.dropna(how="all")

play_col  = 2
route_col = 3

labels = labels[labels[play_col] != "Play"].copy()
labels_df = labels[[play_col, route_col]].copy()
labels_df.columns = ["routeIdx", "routeLabel_raw"]

labels_df["routeIdx"] = pd.to_numeric(labels_df["routeIdx"], errors="coerce")
labels_df = labels_df.dropna(subset=["routeIdx"])
labels_df["routeIdx"] = labels_df["routeIdx"].astype(int)

labels_df["routeLabel_manual"] = (
    labels_df["routeLabel_raw"].astype(str).str.strip().str.lower()
)

coarse_map = {
    "go": "vertical",
    "fade": "vertical",
    "wheel": "vertical",
    "double move": "vertical",

    "in": "in_break",
    "dig": "in_break",
    "slant": "in_break",
    "post": "in_break",
    "over": "in_break",
    "cross": "in_break",

    "out": "out_break",
    "speed out": "out_break",
    "corner": "out_break",
    "flat-l": "out_break",
    "flat-r": "out_break",

    "hitch": "stop",
    "curl": "stop",
    "comeback": "stop",
    "sit": "stop",

    "screen-r": "screen",
    "blocking": "screen",
    "block and go": "screen",
}

labels_df["coarseLabel"] = labels_df["routeLabel_manual"].map(coarse_map)
labels_df = labels_df.dropna(subset=["coarseLabel"])

label_map_idx = dict(
    zip(labels_df["routeIdx"].astype(int), labels_df["coarseLabel"])
)

print("Manual labels loaded:", len(labels_df))
print(labels_df.head())


# ============================================================
# 2) ROUTE SEQUENCE EXTRACTION
# ============================================================
def build_route_sequence(df, gameId, playId, nflId,
                         min_move_x=0.3,
                         min_frames=3):

    sub = df[
        (df["gameId"] == gameId) &
        (df["playId"] == playId) &
        (df["nflId"] == nflId)
    ].sort_values("frameId").copy()

    if sub.empty or len(sub) < min_frames:
        return None

    if "event" in sub.columns:
        snap_rows = sub[sub["event"] == "ball_snap"]
        pass_rows = sub[sub["event"] == "pass_forward"]

        if not snap_rows.empty:
            snap_frame = snap_rows["frameId"].min()
            sub = sub[sub["frameId"] >= snap_frame]

        if not pass_rows.empty:
            pass_frame = pass_rows["frameId"].min()
            sub = sub[sub["frameId"] <= pass_frame]

        if sub.empty or len(sub) < min_frames:
            return None

    x_raw_full = sub["x"].values
    x0_full = x_raw_full[0]
    delta_x = x_raw_full - x0_full

    move_idx = np.where(np.abs(delta_x) >= min_move_x)[0]
    if len(move_idx) > 0:
        start_idx = move_idx[0]
        sub = sub.iloc[start_idx:].copy()
        if sub.empty or len(sub) < min_frames:
            return None

    x_raw = sub["x"].values
    y = sub["y"].values

    x0 = x_raw[0]
    x = x_raw - x0

    dx = np.gradient(x)
    dy = np.gradient(y)
    speed = np.sqrt(dx**2 + dy**2)
    accel = np.gradient(speed)

    if x[-1] < 0:
        x = -x
        dx = -dx
        speed = np.sqrt(dx**2 + dy**2)
        accel = np.gradient(speed)

    if len(x) < min_frames:
        return None

    seq = np.stack([x, y, dx, dy, speed, accel], axis=1)
    return seq


def normalize_length(seq, T=40):
    T0, F = seq.shape
    if T0 == T:
        return seq

    old_idx = np.arange(T0)
    new_idx = np.linspace(0, T0 - 1, T)

    out = np.zeros((T, F), dtype=np.float32)
    for i in range(F):
        out[:, i] = np.interp(new_idx, old_idx, seq[:, i])
    return out


# ============================================================
# 3) DATASET BUILDING
# ============================================================
ALL_T = 40
FEAT_DIM = 6

X = []
y = []

for i, r in route_catalog.iterrows():
    if i not in label_map_idx:
        continue

    g = int(r["gameId"])
    p = int(r["playId"])
    nid = int(r["nflId"])

    seq = build_route_sequence(df, g, p, nid)
    if seq is None:
        continue

    seq_norm = normalize_length(seq, T=ALL_T)
    X.append(seq_norm)
    y.append(label_map_idx[i])

X = np.array(X, dtype=np.float32)
y = np.array(y)

print("Labeled routes used:", X.shape[0])
print("Sequence shape:", X.shape)

le = LabelEncoder()
y_enc = le.fit_transform(y)
num_classes = len(le.classes_)

counts = np.bincount(y_enc)
print("Coarse classes:", list(le.classes_))
print("Counts per class:", dict(zip(le.classes_, counts)))

min_per_class = 2
valid_class_idx = np.where(counts >= min_per_class)[0]
mask_valid = np.isin(y_enc, valid_class_idx)

X_f = X[mask_valid]
y_f = y_enc[mask_valid]

counts_f = np.bincount(y_f)
print("After filtering:", dict((le.classes_[i], int(c)) for i,c in enumerate(counts_f)))





Manual labels loaded: 94
   routeIdx routeLabel_raw routeLabel_manual coarseLabel
5         0           over              over    in_break
6         1           post              post    in_break
7         2          wheel             wheel    vertical
8         3          wheel             wheel    vertical
9         4             go                go    vertical
Labeled routes used: 94
Sequence shape: (94, 40, 6)
Coarse classes: [np.str_('in_break'), np.str_('out_break'), np.str_('screen'), np.str_('stop'), np.str_('vertical')]
Counts per class: {np.str_('in_break'): np.int64(26), np.str_('out_break'): np.int64(32), np.str_('screen'): np.int64(5), np.str_('stop'): np.int64(13), np.str_('vertical'): np.int64(18)}
After filtering: {np.str_('in_break'): 26, np.str_('out_break'): 32, np.str_('screen'): 5, np.str_('stop'): 13, np.str_('vertical'): 18}


In [ ]:
# ============================================================
# DUAL MODELS: Large Dataset (20K) and Small Dataset (100)
# ============================================================
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# ============================================================
# MODEL 1: LARGE DATASET (20,000 routes)
# ============================================================
class LargeDatasetModel(nn.Module):
    """Deep, expressive model for large datasets"""
    def __init__(self, input_dim=6, num_classes=5):
        super().__init__()

        # Multi-scale CNN
        self.stem = nn.Sequential(
            nn.Conv1d(input_dim, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        # Residual blocks with different kernel sizes
        self.blocks = nn.ModuleList([
            self._make_residual_block(64, 128, kernel_size=3),
            self._make_residual_block(128, 256, kernel_size=5),
            self._make_residual_block(256, 256, kernel_size=7),
        ])

        # BiLSTM for temporal patterns
        self.bilstm = nn.LSTM(
            input_size=256, hidden_size=128, num_layers=2,
            batch_first=True, bidirectional=True, dropout=0.3
        )

        # Multi-head attention
        self.attention = nn.MultiheadAttention(
            embed_dim=256, num_heads=8, batch_first=True, dropout=0.2
        )

        # Classification head
        self.fc = nn.Sequential(
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
        )

    def _make_residual_block(self, in_ch, out_ch, kernel_size):
        return nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size, padding=kernel_size//2),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(out_ch, out_ch, kernel_size, padding=kernel_size//2),
            nn.BatchNorm1d(out_ch),
        )

    def forward(self, x):
        # x: (B, T, F)
        x = x.permute(0, 2, 1)  # (B, F, T)
        x = self.stem(x)  # (B, 64, T)

        # Residual blocks
        for block in self.blocks:
            x_res = x
            x = block(x)
            if x.shape == x_res.shape:
                x = x + x_res
            x = F.relu(x)

        # BiLSTM
        x = x.permute(0, 2, 1)  # (B, T, 256)
        x, _ = self.bilstm(x)  # (B, T, 256)

        # Multi-head attention
        x, _ = self.attention(x, x, x)  # (B, T, 256)

        # Global pooling
        x = F.adaptive_avg_pool1d(x.permute(0, 2, 1), 1).squeeze(-1)  # (B, 256)

        # Classification
        logits = self.fc(x)
        return logits


# ============================================================
# MODEL 2: SMALL DATASET (100 routes)
# ============================================================
class SmallDatasetModel(nn.Module):
    """Lightweight TCN ensemble for tiny datasets"""
    def __init__(self, input_dim=6, num_classes=5, num_models=5):
        super().__init__()

        self.models = nn.ModuleList([
            self._make_tcn(input_dim, num_classes)
            for _ in range(num_models)
        ])

    def _make_tcn(self, input_dim, num_classes):
        layers = []
        num_levels = 2
        for i in range(num_levels):
            dilation = 2 ** i
            in_ch = input_dim if i == 0 else 64
            out_ch = 64

            layers.extend([
                nn.Conv1d(in_ch, out_ch, kernel_size=3,
                         padding=(3-1)*dilation, dilation=dilation),
                nn.BatchNorm1d(out_ch),
                nn.ReLU(),
                nn.Dropout(0.4),
            ])

        layers.extend([
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(32, num_classes),
        ])

        return nn.Sequential(*layers)

    def forward(self, x):
        # x: (B, T, F)
        x = x.permute(0, 2, 1)  # (B, F, T)
        logits_list = [model(x) for model in self.models]
        return torch.stack(logits_list).mean(dim=0)  # Average ensemble


# ============================================================
# AUGMENTATION STRATEGIES
# ============================================================
def augment_large(x, device, p=0.7):
    """Moderate augmentation for large datasets"""
    if torch.rand(1, device=device).item() > p:
        return x

    x = x.clone()

    if torch.rand(1, device=device).item() < 0.6:
        noise = 0.02 * torch.randn_like(x)
        x = x + noise

    if torch.rand(1, device=device).item() < 0.5:
        scales = 1.0 + 0.08 * torch.randn(x.shape[0], 1, x.shape[2], device=device)
        x = x * scales.clamp(0.7, 1.3)

    return x


def augment_small(x, device, p=0.95):
    """Aggressive augmentation for small datasets"""
    if torch.rand(1, device=device).item() > p:
        return x

    B, T, F = x.shape
    x = x.clone()

    # High noise
    if torch.rand(1, device=device).item() < 0.9:
        noise = 0.04 * torch.randn_like(x)
        x = x + noise

    # Feature scaling
    if torch.rand(1, device=device).item() < 0.8:
        scales = 1.0 + 0.12 * torch.randn(B, 1, F, device=device)
        x = x * scales.clamp(0.5, 1.5)

    # Temporal warping
    if torch.rand(1, device=device).item() < 0.6:
        warp = 0.9 + 0.2 * torch.rand(1, device=device).item()
        idx = (torch.arange(T, device=device, dtype=torch.float32) * warp).long()
        idx = torch.clamp(idx, 0, T - 1)
        x = x[:, idx, :]

    # Mixup
    if torch.rand(1, device=device).item() < 0.5 and B > 1:
        lam = torch.rand(1, device=device).item()
        idx = torch.randperm(B, device=device)
        x = lam * x + (1 - lam) * x[idx]

    return x


# ============================================================
# TRAINING FUNCTION
# ============================================================
def train_model(X_all, y_all, FEAT_DIM, num_classes, model_type="large", fold_idx=None):
    """
    model_type: "large" (20K routes) or "small" (100 routes)
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"

    if model_type == "large":
        is_large = True
        skf = KFold(n_splits=5, shuffle=True, random_state=42)
    else:
        is_large = False
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_all, y_all if not is_large else None), start=1):
        print(f"\n===== {model_type.upper()} MODEL - Fold {fold} =====")

        X_tr = X_all[train_idx].to(device)
        y_tr = y_all[train_idx].to(device)
        X_val = X_all[val_idx].to(device)
        y_val = y_all[val_idx].to(device)

        # Class weights
        cls_counts = np.bincount(y_tr.cpu().numpy(), minlength=num_classes)
        if is_large:
            weights = torch.tensor(1.0 / (cls_counts + 1e-6), dtype=torch.float32, device=device)
        else:
            weights = torch.tensor(1.0 / np.sqrt(cls_counts + 1), dtype=torch.float32, device=device)
        weights = weights / weights.sum() * num_classes

        # Model
        if is_large:
            model = LargeDatasetModel(input_dim=FEAT_DIM, num_classes=num_classes).to(device)
            batch_size = 64
            epochs = 100
            lr = 5e-4
            label_smooth = 0.1
            scheduler_type = "cosine"
        else:
            model = SmallDatasetModel(input_dim=FEAT_DIM, num_classes=num_classes, num_models=5).to(device)
            batch_size = 8
            epochs = 120
            lr = 1e-3
            label_smooth = 0.2
            scheduler_type = "onecycle"

        loss_fn = nn.CrossEntropyLoss(weight=weights, label_smoothing=label_smooth)
        opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)

        if scheduler_type == "cosine":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
        else:
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                opt, max_lr=2*lr, total_steps=epochs, pct_start=0.3
            )

        train_ds = TensorDataset(X_tr, y_tr)
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        best_val_acc = 0
        best_state = None
        patience = 20
        patience_counter = 0

        for epoch in range(epochs):
            model.train()
            train_loss = 0.0

            for X_batch, y_batch in train_loader:
                opt.zero_grad()
                if is_large:
                    X_aug = augment_large(X_batch, device, p=0.7)
                else:
                    X_aug = augment_small(X_batch, device, p=0.95)

                logits = model(X_aug)
                loss = loss_fn(logits, y_batch)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                opt.step()
                train_loss += loss.item()

            scheduler.step()

            # Validation
            model.eval()
            with torch.no_grad():
                val_logits = model(X_val)
                val_pred = val_logits.argmax(dim=1)
                val_acc = (val_pred == y_val).float().mean().item()

            if (epoch + 1) % 25 == 0 or epoch == 0:
                print(f"  Epoch {epoch+1}: train_loss={train_loss/len(train_loader):.4f}, val_acc={val_acc:.3f}")

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_state = {k: v.cpu() for k, v in model.state_dict().items()}
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"  Early stopping at epoch {epoch+1}")
                    break

        if best_state:
            model.load_state_dict({k: v.to(device) for k, v in best_state.items()})

        print(f"Fold {fold}: best_val_acc = {best_val_acc:.3f}")
        fold_accuracies.append(best_val_acc)

    return fold_accuracies


# ============================================================
# MAIN EXECUTION
# ============================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

X_all = torch.tensor(X_f, dtype=torch.float32)
y_all = torch.tensor(y_f, dtype=torch.long)

print(f"Dataset: {X_all.shape[0]} samples, {X_all.shape[1]} timesteps, {X_all.shape[2]} features")
print(f"Class distribution: {np.bincount(y_f)}\n")

# Determine which model to train
dataset_size = X_all.shape[0]

if dataset_size >= 10000:
    print("TRAINING LARGE DATASET MODEL (for 20K routes)")
    fold_accs = train_model(X_all, y_all, FEAT_DIM, num_classes, model_type="large")
    print("\n" + "="*60)
    print(f"LARGE MODEL - CV Accuracies: {[round(a, 3) for a in fold_accs]}")
    print(f"Mean: {round(float(np.mean(fold_accs)), 3)}, Std: {round(float(np.std(fold_accs)), 3)}")
    print("="*60)

else:
    print("TRAINING SMALL DATASET MODEL (for 100 routes)")
    fold_accs = train_model(X_all, y_all, FEAT_DIM, num_classes, model_type="small")
    print("\n" + "="*60)
    print(f"SMALL MODEL - CV Accuracies: {[round(a, 3) for a in fold_accs]}")
    print(f"Mean: {round(float(np.mean(fold_accs)), 3)}, Std: {round(float(np.std(fold_accs)), 3)}")
    print("="*60)


# ============================================================
# FINAL TRAINING ON ALL DATA (select model based on dataset size)
# ============================================================
print("\nTraining final model on all data...")

if dataset_size >= 10000:
    model = LargeDatasetModel(input_dim=FEAT_DIM, num_classes=num_classes).to(device)
    batch_size = 64
    epochs = 100
    lr = 5e-4
    augment_fn = augment_large
else:
    model = SmallDatasetModel(input_dim=FEAT_DIM, num_classes=num_classes, num_models=5).to(device)
    batch_size = 8
    epochs = 120
    lr = 1e-3
    augment_fn = augment_small

cls_counts_full = np.bincount(y_all.cpu().numpy(), minlength=num_classes)
if dataset_size >= 10000:
    weights_full = torch.tensor(1.0 / (cls_counts_full + 1e-6), dtype=torch.float32, device=device)
    label_smooth = 0.1
else:
    weights_full = torch.tensor(1.0 / np.sqrt(cls_counts_full + 1), dtype=torch.float32, device=device)
    label_smooth = 0.2

weights_full = weights_full / weights_full.sum() * num_classes

loss_fn = nn.CrossEntropyLoss(weight=weights_full, label_smoothing=label_smooth)
opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)

train_ds = TensorDataset(X_all.to(device), y_all.to(device))
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        opt.zero_grad()
        X_aug = augment_fn(X_batch, device, p=0.8 if dataset_size >= 10000 else 0.95)
        logits = model(X_aug)
        loss = loss_fn(logits, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        opt.step()
        train_loss += loss.item()

    scheduler.step()

    if (epoch + 1) % 25 == 0:
        print(f"[FINAL] Epoch {epoch+1}/{epochs} loss={train_loss/len(train_loader):.4f}")


# ============================================================
# PREDICTIONS
# ============================================================
def predict_coarse_route_for_row(row):
    g = int(row["gameId"])
    p = int(row["playId"])
    nid = int(row["nflId"])
    seq = build_route_sequence(df, g, p, nid)
    if seq is None:
        return None
    seq_norm = normalize_length(seq, T=ALL_T)[None, :, :]
    x_t = torch.tensor(seq_norm, dtype=torch.float32).to(device)
    model.eval()
    with torch.no_grad():
        logits = model(x_t)
        idx = logits.argmax(dim=1).item()
    coarse_label = le.inverse_transform([idx])[0]
    return coarse_label


route_catalog["predictedRouteCoarse"] = route_catalog.apply(
    predict_coarse_route_for_row, axis=1
)

print("\nPredictions:")
print(route_catalog[[
    "gameId", "playId", "nflId", "displayName", "predictedRouteCoarse"
]].head(10))

Using device: cuda
Dataset: 94 samples, 40 timesteps, 6 features
Class distribution: [26 32  5 13 18]

TRAINING SMALL DATASET MODEL (for 100 routes)

===== SMALL MODEL - Fold 1 =====
  Epoch 1: train_loss=1.7105, val_acc=0.263
  Epoch 25: train_loss=1.6206, val_acc=0.368
  Early stopping at epoch 33
Fold 1: best_val_acc = 0.526

===== SMALL MODEL - Fold 2 =====
  Epoch 1: train_loss=1.6875, val_acc=0.316
  Epoch 25: train_loss=1.6182, val_acc=0.316
  Early stopping at epoch 27
Fold 2: best_val_acc = 0.421

===== SMALL MODEL - Fold 3 =====
  Epoch 1: train_loss=1.6872, val_acc=0.263
  Epoch 25: train_loss=1.6214, val_acc=0.263
  Early stopping at epoch 43
Fold 3: best_val_acc = 0.474

===== SMALL MODEL - Fold 4 =====
  Epoch 1: train_loss=1.6507, val_acc=0.263
  Epoch 25: train_loss=1.5911, val_acc=0.263
  Early stopping at epoch 33
Fold 4: best_val_acc = 0.474

===== SMALL MODEL - Fold 5 =====
  Epoch 1: train_loss=1.6560, val_acc=0.222
  Epoch 25: train_loss=1.6433, val_acc=0.278
  Ea